<a href="https://colab.research.google.com/github/LeonardoGoncRibeiro/06_MachineLearning/blob/main/02_Advanced/07_NLP_Regex_and_Language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP: Regex and language models

In this course, we will continue our studies in NLP. We will learn how Regex may assist in the treatment of text data. Also, we will understand what are language models and their applications, and we will build a model to detect languages in an automatized way. 

In this course, we will use the following packages:

In [406]:
import pandas as pd
import numpy as np
import nltk

from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Laplace

from sklearn.model_selection import train_test_split

import re

Also, we will use the following datasets:

In [327]:
df_pt = pd.read_csv("stackoverflow_portugues.csv")

In [328]:
df_es = pd.read_csv("stackoverflow_espanhol.csv", encoding = 'latin-1', sep = ';')

In [329]:
df_en = pd.read_csv("stackoverflow_ingles.csv")

Let's check our data:

In [330]:
df_pt.head( )

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [331]:
df_pt['Questão'].loc[0]

'<p>Se eu fizer o <em><a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">hash</a></em> de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?</p>\n\n<p>Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como <a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">força bruta</a> na página de login da aplicação, <em><a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">keylogger</a></em> no cliente e <a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">criptoanálise <em>rubberhose</em></a>. Qualquer forma de <em>hash</em> não vai impedir esses ataques.</p>\n\n<p>Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste as

Note that, since this dataset comes from HTML, it has a lot of unimportant data, such as HTML tags. In our project, we will try to make a language detection algorithm. Thus, we need to treat our data, excluding everything that seems unnecessary. For that end, we will use Regular Expressions (Regex).

# Regular Expressions (Regex)

A Regular Expression (Regex) helps to define patterns in our data so that we may extract important information. For instance, if we have data with multiple e-mails, and we want to store the possible e-mail servers, we should try to find a pattern in our data, such as:

*   Look for everything after @
*   Stop at .com

This pattern is seem in basically all e-mail providers, and would serve to solve our problem. 

We can test our regex code using https://regex101.com/.

Usign regex, we can use meta-characters to pass what we really want to do. Some important metacharacters are:

*   . - can be interpreted as anything.
*   \ - The next metacharacter will not be understood as a metacharacter (for regex).
*   () - Creates a group of words.
*   [] - Creates a group of characters.
*   \* - States that the previous character can be repeated 0 to $n$ times.
*   \+ - States that the previous character can be repeated 1 to $n$ times.
*   {i, j} - States that the previous character can be repeated i to $j$ times.
*   ? - States that the previous character can be repeated 0 or 1 time.
*   ^ - States that the string must start with the following character (e.g. ^t, string should start with t).
*   \$ - States that the string must end with the previous character.
*   | - "or" operator.



# Using Regex in Python

So, here, we will use Regex to filter our data, so that we only get the desired text. For intance, let's get an example of each language:

In [332]:
print(df_pt["Questão"].iloc[0])

<p>Se eu fizer o <em><a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">hash</a></em> de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?</p>

<p>Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como <a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">força bruta</a> na página de login da aplicação, <em><a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">keylogger</a></em> no cliente e <a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">criptoanálise <em>rubberhose</em></a>. Qualquer forma de <em>hash</em> não vai impedir esses ataques.</p>

<p>Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto

In [333]:
print(df_es["Questão"].iloc[0])

<p>Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:</p>

<pre><code>$id_usuario = $_POST["id"];

mysql_query("SELECT * FROM usuarios WHERE id = $id_usuario");
</code></pre>

<p>Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como <code>1; DROP TABLE usuarios;--</code> nos encontraríamos con que la sentencia ejecutada sería:</p>

<pre><code>SELECT * FROM usuarios WHERE id = 1; DROP TABLE usuarios;--
</code></pre>

<p>Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. </p>

<p><strong>¿Cómo puedo evitar que la inyección SQL ocurra en PHP?</strong></p>



In [334]:
print(df_en["Questão"].iloc[0])

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;

    std::cout &lt;&lt; elapsedTime &lt;&lt; std::endl;
    std::cout &lt

## Removing HTML tags

Note that we have a piece of text full of HTML tags. In some cases, we also have some lines of code. Thus, we need to create a regex to identify these patterns, and extract important information from our data. Here, we will use the ```re``` package. Let's get our portuguese question, and try to make a regex. We have a lot of text under a \<p\> <\p> tag. First, let's try to get those:

In [335]:
q_pt = df_pt["Questão"].iloc[0]
re.findall(r"<p>",q_pt)

['<p>', '<p>', '<p>', '<p>']

So, we found 4 tags. Now, let's try to find all tags in our text:

In [336]:
q_pt = df_pt["Questão"].iloc[0]
re.findall(r"<.*?>",q_pt)

['<p>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">',
 '</a>',
 '</em>',
 '</p>',
 '<p>',
 '<a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">',
 '</a>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">',
 '</a>',
 '</em>',
 '<a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">',
 '<em>',
 '</em>',
 '</a>',
 '<em>',
 '</em>',
 '</p>',
 '<p>',
 '</p>',
 '<p>',
 '<em>',
 '</em>',
 '</p>']

This regex seems to solve our problems. Thus, let's compile it:

In [337]:
regex_search_tags = re.compile(r"<.*?>")

Compiling the regex saves us time when applying functions to the regex. Finally, we can simply substitute them for "":

In [338]:
regex_search_tags.sub("",q_pt)

'Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?\n\nEstou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.\n\nTenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?\n\nQuais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?\n'

So, let's create a function to remove the HTML tags from a text:

In [339]:
def remove_HTML_tags(text, regex):
  return regex.sub("", text)

Now, let's test our function:

In [340]:
q_pt = df_pt["Questão"].iloc[0]
print(remove_HTML_tags(q_pt, regex_search_tags))

Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?

Estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.

Tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?

Quais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?



In [341]:
q_es = df_es["Questão"].iloc[0]
print(remove_HTML_tags(q_es, regex_search_tags))

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:

$id_usuario = $_POST["id"];

mysql_query("SELECT * FROM usuarios WHERE id = $id_usuario");


Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como 1; DROP TABLE usuarios;-- nos encontraríamos con que la sentencia ejecutada sería:

SELECT * FROM usuarios WHERE id = 1; DROP TABLE usuarios;--


Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. 

¿Cómo puedo evitar que la inyección SQL ocurra en PHP?



In [342]:
q_en = df_en["Questão"].iloc[0]
print(remove_HTML_tags(q_es, regex_search_tags))

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:

$id_usuario = $_POST["id"];

mysql_query("SELECT * FROM usuarios WHERE id = $id_usuario");


Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como 1; DROP TABLE usuarios;-- nos encontraríamos con que la sentencia ejecutada sería:

SELECT * FROM usuarios WHERE id = 1; DROP TABLE usuarios;--


Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. 

¿Cómo puedo evitar que la inyección SQL ocurra en PHP?



## Removing code

So, we managed to remove our HTML tags. However, note that our text still has code inside it. We can use the HTML tags to identify the code, and try to remove those first. Then, we can remove the HTML tags. Let's try to create a regex to identify the code and substitute it with the word CODE:

In [343]:
print(q_en)

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;

    std::cout &lt;&lt; elapsedTime &lt;&lt; std::endl;
    std::cout &lt

In [344]:
print(re.sub(r"<code>(.|(\n))*?</code>","CODE", q_en))

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override">CODE</pre>

<ul>
<li>Without CODE, the code runs in 11.54 seconds.</li>
<li>With the sorted data, the code runs in 1.93 seconds.</li>
</ul>

<p>Initially, I thought this might be just a language or compiler anomaly. So I tried it in Java.</p>

<pre class="lang-java prettyprint-override">CODE</pre>

<p>With a somewhat similar but less extreme result.</p>

<hr>

<p>My first thought was that sorting brings the data into the cache, but then I thought how silly that is because the array was just generated.</p>

<ul>
<li>What is going on?</li>
<li>Why is it faster to process a sorted array than an unsorted array?</li>
<li>The code is summing up some independent terms, and the order should not matter.</li>
</ul>



Nice! It seems to have worked out. Let's modify our user defined function:

In [345]:
regex_code = re.compile(r"<code>(.|(\n))*?</code>")

In [346]:
def get_important_text(text, regex, regex_code, code_substitution):
  text_no_code = regex_code.sub("CODE", text)
  return regex.sub("", text_no_code)

Let's test it:

In [347]:
print(get_important_text(q_en, regex_search_tags, regex_code, "CODE"))

Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.

CODE


Without CODE, the code runs in 11.54 seconds.
With the sorted data, the code runs in 1.93 seconds.


Initially, I thought this might be just a language or compiler anomaly. So I tried it in Java.

CODE

With a somewhat similar but less extreme result.



My first thought was that sorting brings the data into the cache, but then I thought how silly that is because the array was just generated.


What is going on?
Why is it faster to process a sorted array than an unsorted array?
The code is summing up some independent terms, and the order should not matter.




Nice! It worked out. 

Now, let's apply the regex to our entire dataset:

In [348]:
df_pt["Q_Treat"] = df_pt["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_code, "CODE"))

df_en["Q_Treat"] = df_en["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_code, "CODE"))

df_es["Q_Treat"] = df_es["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_code, "CODE"))

Let's see some examples to guarantee that everything worked out:

In [349]:
print(df_pt["Q_Treat"].iloc[12])

Estou pensando em fazer uma aplicação para vender, gostaria de saber como proteger meu código fonte para manter meu software seguro.

Eu vi que os bytecodes do Java, armazenados no arquivo CODE são facilmente convertidos de volta para CODE usando a ferramenta Java Decompiler

Os CODE podem ser facilmente encontrados dentro de um CODE para o caso dos desktops, ou até mesmo em um CODE, conforme mostrado nesse post.

Como fazem os milhões de aplicativos no Google Play para protegerem seus códigos fontes? Pois deixá-los expostos me parece meio inseguro, ou não é? Se sim (se é inseguro), tem como eu desenvolver aplicações que mantenham escondido o código fonte em Java para desktop e Android?

EDIÇÃO

A resposta do @mgibsonbr me convenceu de não ser paranoico e querer proteger todo o código fonte, entretanto existem alguns pontos da aplicação que não podem ser revelados por motivo de segurança dos dados, por exemplo:

CODE

Imaginem o estrago se alguém tiver o endereço, o usuário e a senha d

In [350]:
print(df_en["Q_Treat"].iloc[5])

What is the use of the CODE keyword in Python? What does it do?

For example, I'm trying to understand this code1:

CODE

And this is the caller:

CODE

What happens when the method CODE is called?
Is a list returned? A single element? Is it called again? When will subsequent calls stop?




1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to the complete source: Module mspace.



In [351]:
print(df_es["Q_Treat"].iloc[5])

Siempre he visto que en CODE hay:


asignaciones CODE
comparaciones CODE y CODE


Creo entender que CODE hace algo parecido a comparar el valor de la variable y el CODE también compara el tipo (como un equals de java). 

¿Alguien podría confirmarme este punto y extenderlo?. Soy javero y el no tipado de javascript a veces me encanta y otras lo odio.



¿Cuál es la manera correcta en javascript de comparar CODE, CODE y otros valores por defecto? 

CODE

¿CODE se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento CODE sin CODE? (por ejemplo un label sin contenido)

CODE



Nice! It seems to have worked out fine.

# Removing digits and punctuation

We want to create a model that allow us to detect which language is being used. Thus, we need to be able to read the WORDS, and nothing else. Things such as digits and punctuation are common for every language and, thus, may make it harder for our model to detect the language. First, let's use a regex to obtain our non-alphanumeric characthers:

In [352]:
regex_punct = re.compile(r"[^\w\s]")  

Here, we are getting everything that is not "\w" (alphanumeric) or "\s" (space). Let's try to use it:

In [353]:
print(regex_punct.sub("", q_en))

pHere is a piece of C code that seems very peculiar For some strange reason sorting the data miraculously makes the code almost six times fasterp

pre classlangcpp prettyprintoverridecodeinclude ltalgorithmgt
include ltctimegt
include ltiostreamgt

int main

     Generate data
    const unsigned arraySize  32768
    int dataarraySize

    for unsigned c  0 c lt arraySize c
        datac  stdrand  256

      With this the next loop runs faster
    stdsortdata data  arraySize

     Test
    clock_t start  clock
    long long sum  0

    for unsigned i  0 i lt 100000 i
    
         Primary loop
        for unsigned c  0 c lt arraySize c
        
            if datac gt 128
                sum  datac
        
    

    double elapsedTime  static_castltdoublegtclock  start  CLOCKS_PER_SEC

    stdcout ltlt elapsedTime ltlt stdendl
    stdcout ltlt sum   ltlt sum ltlt stdendl

codepre

ul
liWithout codestdsortdata data  arraySizecode the code runs in 1154 secondsli
liWith the sorted data th

Nice! Also, the last thing we may implement is a function that turns everything into lowercase. We can do that simply using the ```lower( )``` method.



Now, let's implement this again on our function:

In [354]:
def get_important_text(text, regex, regex_punct, regex_code, code_substitution):
  text_no_code = regex_code.sub(code_substitution, text)
  text_no_tags = regex.sub("", text_no_code)
  return regex_punct.sub("", text_no_tags).lower( )

Finally, let's update our treatment:

In [355]:
df_pt["Q_Treat"] = df_pt["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_punct, regex_code, "CODE"))

df_en["Q_Treat"] = df_en["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_punct, regex_code, "CODE"))

df_es["Q_Treat"] = df_es["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_punct, regex_code, "CODE"))

Now, we can check again if everything went alright:

In [356]:
print(df_pt["Q_Treat"].iloc[12])

estou pensando em fazer uma aplicação para vender gostaria de saber como proteger meu código fonte para manter meu software seguro

eu vi que os bytecodes do java armazenados no arquivo code são facilmente convertidos de volta para code usando a ferramenta java decompiler

os code podem ser facilmente encontrados dentro de um code para o caso dos desktops ou até mesmo em um code conforme mostrado nesse post

como fazem os milhões de aplicativos no google play para protegerem seus códigos fontes pois deixálos expostos me parece meio inseguro ou não é se sim se é inseguro tem como eu desenvolver aplicações que mantenham escondido o código fonte em java para desktop e android

edição

a resposta do mgibsonbr me convenceu de não ser paranoico e querer proteger todo o código fonte entretanto existem alguns pontos da aplicação que não podem ser revelados por motivo de segurança dos dados por exemplo

code

imaginem o estrago se alguém tiver o endereço o usuário e a senha do seu banco de dado

In [357]:
print(df_en["Q_Treat"].iloc[5])

what is the use of the code keyword in python what does it do

for example im trying to understand this code1

code

and this is the caller

code

what happens when the method code is called
is a list returned a single element is it called again when will subsequent calls stop




1 this piece of code was written by jochen schulz jrschulz who made a great python library for metric spaces this is the link to the complete source module mspace



In [358]:
print(df_es["Q_Treat"].iloc[5])

siempre he visto que en code hay


asignaciones code
comparaciones code y code


creo entender que code hace algo parecido a comparar el valor de la variable y el code también compara el tipo como un equals de java 

alguien podría confirmarme este punto y extenderlo soy javero y el no tipado de javascript a veces me encanta y otras lo odio



cuál es la manera correcta en javascript de comparar code code y otros valores por defecto 

code

code se usa como cadena de texto o como palabra clave cual de las siguientes comparaciones es la correcta para un elemento code sin code por ejemplo un label sin contenido

code



Now, our text is in lowercase and there are no punctuation marks! Finally, we can remove our digits using another regex method:

In [359]:
regex_digits = re.compile(r"\d+")

Updating our function:

In [360]:
def get_important_text(text, regex, regex_punct, regex_digits, regex_code, code_substitution):
  text_no_code = regex_code.sub(code_substitution, text)
  text_no_tags = regex.sub("", text_no_code)
  text_no_punct = regex_punct.sub("", text_no_tags)
  text_no_digits = regex_digits("", text_no_punct)
  return text_no_digits.lower( )

One final thing: let's remove duplicated spaces. We can simply do:

In [361]:
regex_dup_spaces = re.compile(r" +")
regex_line_break = re.compile(r"(\n)")

In [362]:
def get_important_text(text, regex, regex_punct, regex_digits, regex_dup_spaces, regex_line_break, regex_code, code_substitution):
  # Substituting code
  text_no_code = regex_code.sub(code_substitution, text)
  # Removing HTML tags
  text_no_tags = regex.sub("", text_no_code)
  # Removing punctuation
  text_no_punct = regex_punct.sub("", text_no_tags)
  # Removing decimal digits
  text_no_digits = regex_digits.sub("", text_no_punct)
  # Removing line breaks
  text_no_line_break = regex_line_break.sub(" ", text_no_digits)
  # Removing duplicated spaces
  text_no_dup_space = regex_dup_spaces.sub(" ", text_no_line_break)
  return text_no_dup_space.lower( )

# Treating our dataset

Finally, we have implemented all of our Regular Expressions in a single function. Now, let's performing the treatment in the portuguese dataset:

In [363]:
df_pt["Q_Treat"] = df_pt["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_punct, regex_digits, regex_dup_spaces, regex_line_break, regex_code, "CODE"))

And, if we check an example, we get:

In [364]:
print(df_pt["Q_Treat"].iloc[12])

estou pensando em fazer uma aplicação para vender gostaria de saber como proteger meu código fonte para manter meu software seguro eu vi que os bytecodes do java armazenados no arquivo code são facilmente convertidos de volta para code usando a ferramenta java decompiler os code podem ser facilmente encontrados dentro de um code para o caso dos desktops ou até mesmo em um code conforme mostrado nesse post como fazem os milhões de aplicativos no google play para protegerem seus códigos fontes pois deixálos expostos me parece meio inseguro ou não é se sim se é inseguro tem como eu desenvolver aplicações que mantenham escondido o código fonte em java para desktop e android edição a resposta do mgibsonbr me convenceu de não ser paranoico e querer proteger todo o código fonte entretanto existem alguns pontos da aplicação que não podem ser revelados por motivo de segurança dos dados por exemplo code imaginem o estrago se alguém tiver o endereço o usuário e a senha do seu banco de dados nos c

Nice! This is very good: we managed to simply keep the portuguese words in our text, with no punctuation, digits, or line breaks. This will be very good when we implement our language detection model. 

Now, let's apply the treatment in the other dataframes:

In [365]:
df_en["Q_Treat"] = df_en["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_punct, regex_digits, regex_dup_spaces, regex_line_break, regex_code, "CODE"))

df_es["Q_Treat"] = df_es["Questão"].apply(lambda x : get_important_text(x, regex_search_tags, regex_punct, regex_digits, regex_dup_spaces, regex_line_break, regex_code, "CODE"))

Finally, now that we have treated our datasets, we can start to create our language model.

# Natural Language Toolkit (NLTK) and Machine Learning

Language models are important tools for Natural Language Processing. To detect the language of our text (portuguese, english, or spanish), we will basically evaluate what is the probability that a certain word is from a given language.

Here, we will use bigrams, which are given by pairs of words taken from a sequence of text. For instance, the text:

* Test.

can be separated into the following biagrams:

* (T, e), (e, s), (s, t)

Let's get an example of bigrams:

In [366]:
list(nltk.bigrams('Test'))

[('T', 'e'), ('e', 's'), ('s', 't')]

Note that the first and final letters are counted only once, while the others are counted twice. This can mess with the probabilities in our model. To fix this, we can use "fake characters". We can do this with:

In [367]:
list(nltk.bigrams(pad_both_ends('Test', n = 2)))

[('<s>', 'T'), ('T', 'e'), ('e', 's'), ('s', 't'), ('t', '</s>')]

Nice! So, here, we used fake chars, forcing that all letters appear twice. Also, our model will be able to understand when is the start and end of each word.

In our model, we need to have explicative features (which is our text) and target variables (which is the language). Let's create a column with the language of each df:

In [368]:
df_pt['Language'] = 'PT'
df_en['Language'] = 'EN'
df_es['Language'] = 'ES'

Now, let's create a single dataset with all of our data:

In [369]:
df_aux1 = df_pt[['Q_Treat', 'Language']].copy( )
df_aux2 = df_en[['Q_Treat', 'Language']].copy( )
df_aux3 = df_es[['Q_Treat', 'Language']].copy( )

Nice. Now, let's divide our dataset into explicative and target features:

In [370]:
X = df_aux1['Q_Treat']
y = df_aux1['Language']

Now, we may perform the train-test split:

In [371]:
X_train_pt, X_test_pt, y_train_pt, y_test_pt = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

## Defining a model to detect the language of a given word

Nice! Now, note that our explicative features are a set of words (which form a text). However, in our language detection model, we want to detect the language of a single word. Thus, we have to separate our text data into multiple words, and define our bigrams for each word.

First, let's get all portuguese text into one feature:

In [372]:
pt_text_train = ' '.join(X_train_pt)

In [373]:
pt_text_train

'ultimamente ouvi muito a respeito de web service web service é uma solução utilizada na integração de sistemas e na comunicação entre aplicações diferentes permitem às aplicações enviar e receber dados em formato xml web service é uma solução utilizada na integração de sistemas e na comunicação entre aplicações diferentes com esta tecnologia é possível que novas aplicações possam interagir com aquelas que já existem e que sistemas desenvolvidos em plataformas diferentes sejam compatíveis os web services são componentes que permitem às aplicações enviar e receber dados em formato xml cada aplicação pode ter a sua própria linguagem que é traduzida para uma linguagem universal o formato xml fonte httpswwwoficinadanetcombrartigoo_que_e_web_service eu não consegui visualizar o que seria um web service mas afinal o que ele é alguém poderia elucidar com alguma aplicação que ele possa ser utilizado talvez isso melhore no meu entendimento  ao criar um componente usamos code para identificálo p

Now, to split our words, we can use a vectorizer:

In [374]:
tokenizer = nltk.tokenize.WhitespaceTokenizer( )
all_pt_words = tokenizer.tokenize(pt_text_train)

In [375]:
all_pt_words

['ultimamente',
 'ouvi',
 'muito',
 'a',
 'respeito',
 'de',
 'web',
 'service',
 'web',
 'service',
 'é',
 'uma',
 'solução',
 'utilizada',
 'na',
 'integração',
 'de',
 'sistemas',
 'e',
 'na',
 'comunicação',
 'entre',
 'aplicações',
 'diferentes',
 'permitem',
 'às',
 'aplicações',
 'enviar',
 'e',
 'receber',
 'dados',
 'em',
 'formato',
 'xml',
 'web',
 'service',
 'é',
 'uma',
 'solução',
 'utilizada',
 'na',
 'integração',
 'de',
 'sistemas',
 'e',
 'na',
 'comunicação',
 'entre',
 'aplicações',
 'diferentes',
 'com',
 'esta',
 'tecnologia',
 'é',
 'possível',
 'que',
 'novas',
 'aplicações',
 'possam',
 'interagir',
 'com',
 'aquelas',
 'que',
 'já',
 'existem',
 'e',
 'que',
 'sistemas',
 'desenvolvidos',
 'em',
 'plataformas',
 'diferentes',
 'sejam',
 'compatíveis',
 'os',
 'web',
 'services',
 'são',
 'componentes',
 'que',
 'permitem',
 'às',
 'aplicações',
 'enviar',
 'e',
 'receber',
 'dados',
 'em',
 'formato',
 'xml',
 'cada',
 'aplicação',
 'pode',
 'ter',
 'a',
 'su

Nice! Now, let's add our fake chars:

In [376]:
all_pt_bigrams, vocab_pt = padded_everygram_pipeline(2, all_pt_words)

Now, let's do a similar thing for the other two languages:

In [377]:
X = df_aux2['Q_Treat']
y = df_aux2['Language']
X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)
all_en_text = ' '.join(X_train_en)
all_en_words = tokenizer.tokenize(all_en_text)
all_en_bigrams, vocab_en = padded_everygram_pipeline(2, all_en_words)

In [378]:
X = df_aux3['Q_Treat']
y = df_aux3['Language']
X_train_es, X_test_es, y_train_es, y_test_es = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)
all_es_text = ' '.join(X_train_es)
all_es_words = tokenizer.tokenize(all_en_text)
all_es_bigrams, vocab_es = padded_everygram_pipeline(2, all_es_words)

Now, we can fit an MLE model to detect the probability that a given word is from the portuguese language:

In [379]:
model_pt = nltk.lm.MLE(2)
model_pt.fit(all_pt_bigrams, vocab_pt)

## Perplexity

We can evaluate the probability that a given word is from a given language. In our case, this will be represented by the perplexity. For instance:

In [382]:
def GetTransformedWord(word):
  return nltk.bigrams(pad_both_ends(word.lower( ), n = 2))

In [383]:
Word = "Bom"
new_word = GetTransformedWord(Word)
model_pt.perplexity(new_word)

15.06440684750102

Nice!

## Obtaining the other models

Finally, let's create an user defined function to fit our model. Then, we will create a function to evaluate the perplexity for a given word. Finally, we will create a function to get the perplexity of an entire phrase.

Let's start:

In [407]:
def FitMLEModel(X_train):
  all_text = ' '.join(X_train)
  all_words = tokenizer.tokenize(all_text)
  all_bigrams, vocab = padded_everygram_pipeline(2, all_words)

  model = MLE(2)
  model.fit(all_bigrams, vocab)

  return model

Now, a function to return the perplexity of a given word:

In [387]:
def GetPerplexity(model, word):
  word_aux = GetTransformedWord(word)
  return model.perplexity(word_aux)

Let's test with our portuguese model:

In [388]:
model_pt = FitMLEModel(X_train_pt)
perp_pt = GetPerplexity(model_pt, "Bom")

In [389]:
perp_pt

15.06440684750102

Nice! We got the same perplexity. Now, we can create our other models:

In [393]:
model_en = FitMLEModel(X_train_en)
perp_en = GetPerplexity(model_en, "Bom")
perp_en

13.630990439787295

In [392]:
model_es = FitMLEModel(X_train_es)
perp_es = GetPerplexity(model_es, "Bom")
perp_es

43.13441134592497

Note that the perplexity for the english model was actually lower! Thus, the model thinks that the word "Bom" is from the english language. 

## Perplexity for an entire text

Now, let's create a function to obtain the perplexity for an entire phrase:

In [398]:
def GetPerplexityText(model, text):
  p = 0

  words = nltk.tokenize.WhitespaceTokenizer( ).tokenize(text)

  for word in words:
    p += GetPerplexity(model, word)

  return p

Nice! Now, let's get a portuguese text, and check its perplexity:

In [399]:
pt_test = df_aux1.Q_Treat.iloc[0]
pt_test

'se eu fizer o hash de senhas antes de armazenálas em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque como força bruta na página de login da aplicação keylogger no cliente e criptoanálise rubberhose qualquer forma de hash não vai impedir esses ataques tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido como dar maior garantia de segurança neste aspecto quais preocupações adicionais evitariam o acesso às senhas existem formas melhores de fazer esse hash '

In [402]:
GetPerplexityText(model_pt, pt_test)

1201.0271279239876

Now, let's evaluate the perplexity using our other models:

In [403]:
GetPerplexityText(model_en, pt_test)

inf

In [404]:
GetPerplexityText(model_es, pt_test)

inf

Nice! Since the perplexity for the portuguese model was the lowest one, it worked! We were able to guess that the text is from the portuguese language.

## Infinite perplexity

Note that, for the text, we actually got an infinite perplexity for the english and spanish language. This means that, according to our model, it is impossible that our text is from the english language. Actually, this is an error of our model: nothing should be impossible, but rather very difficult. However, this occurs because we trained using a limited number of words. 

What our model essentially do is check the set of bigrams, and evaluate the probability that those sets of biagram exist in the language. However, if the model does not see a given bigram, it we think that the probability of it happening is null. Thus, the perplexity is infinite!

To solve this, we can use different models, such as the Laplace model.

# Laplace model

To solve the infinite perplexity problem, we should simply make it impossible to have a probability of zero. To do so, we can use a Laplace smoothing. Thus, let's try to use it:

In [408]:
def FitLaplaceModel(X_train):
  all_text = ' '.join(X_train)
  all_words = tokenizer.tokenize(all_text)
  all_bigrams, vocab = padded_everygram_pipeline(2, all_words)

  model = Laplace(2)
  model.fit(all_bigrams, vocab)

  return model

Nice! Now, let's fit all of our models, and evaluate their perplexity again:

In [409]:
model_pt = FitLaplaceModel(X_train_pt)
model_en = FitLaplaceModel(X_train_en)
model_es = FitLaplaceModel(X_train_es)

In [410]:
GetPerplexityText(model_pt, pt_test)

1203.1552501461438

In [411]:
GetPerplexityText(model_en, pt_test)

3159.7754120899795

In [412]:
GetPerplexityText(model_es, pt_test)

2173.2287167447803

Nice! Now, due to our smoothing, we didn't get an infinite perplexity and, still, our portuguese model showed the lowest perplexity. 

# Obtaining the language of our text

Let's define a function that returns the language for a given text:

In [443]:
def GetLanguage(text):
  language = ""

  perp_pt = GetPerplexityText(model_pt, text)
  perp_en = GetPerplexityText(model_en, text)
  perp_es = GetPerplexityText(model_es, text)

  perp_array = np.array([perp_pt, perp_en, perp_es])
  min_ind = np.argmin(perp_array)

  if min_ind == 0:
    language = "PT"
  elif min_ind == 1:
    language = "EN"
  elif min_ind == 2:
    language = "ES"
  else:
    language = "ERROR"

  return language

Let's test this function:

In [444]:
GetLanguage(df_aux1.Q_Treat.iloc[2])

'PT'

Nice! Now, let's test our language detection algorithm in our test data:

In [445]:
df_aux1["Detected"] = df_aux1.Q_Treat.apply(GetLanguage)

Let's see our results:

In [446]:
df_aux1.Detected.value_counts( )

PT    497
ES      2
EN      1
Name: Detected, dtype: int64

So, we only missed 3 entries here. We can get the accuracy from:

In [451]:
acc = sum(df_aux1.Detected == df_aux1.Language)/len(df_aux1)*100
print("Accuracy: {:.2f}%".format(acc))

Accuracy: 99.40%


Let's define a function to get this:

In [455]:
def GetModelAccuracy(df):
  df_new = df.copy( )
  df_new["Detected"] = df_new.Q_Treat.apply(GetLanguage)
  acc = sum(df_new.Detected == df_new.Language)/len(df_new)*100
  return df_new, acc

Now, we can evaluate the accuracy for the other two datasets:

In [456]:
# English
df_aux2, acc = GetModelAccuracy(df_aux2)
print("Accuracy: {:.2f}%".format(acc))

Accuracy: 100.00%


In [457]:
# Spanish
df_aux3, acc = GetModelAccuracy(df_aux3)
print("Accuracy: {:.2f}%".format(acc))

Accuracy: 99.00%


So, we got a 100% accuracy in the english dataset, and a 99% accuracy in the spanish dataset. Nice!